# VESU Protocol Analysis

VESU is a lending protocol deployed on Starknet that enables users to deposit assets as collateral and borrow other assets. This notebook analyzes key aspects of the protocol:

- **Health Factor**: Measures the safety of a position against liquidation for a specific pool.
- **Interest Rate Model**: Determines how borrowing and lending rates are calculated based on utilization.
- **Deposit Functionality**: Evaluates supported assets and their total supply in a pool.
- **Borrowing Potential**: Calculates the maximum borrowing capacity for a given collateral.

Contract address: `0x02545b2e5d519fc230e9cd781046d3a64e092114f07e44771e0d719d148725ef`

## Protocol Technical Architecture

The VESU protocol implements an over-collateralized lending model with the following components:

1. **Asset Configuration**: Configuration parameters for each supported asset are stored in the `asset_config` mapping.
2. **Collateralization Rules**: Loan-to-Value (LTV) ratios are defined in the `ltv_config` mapping.
3. **Interest Rate Model**: Dynamic rates are based on utilization, with separate borrow and supply rates.
4. **Liquidation Mechanism**: Positions with a health factor below 1 are subject to liquidation.

This notebook uses the Starknet client to interact with these components and fetch on-chain data.

## Setup and Imports

This section initializes the required libraries, defines contract addresses, and sets up the Starknet client for blockchain interaction.

In [25]:
import sys
import asyncio
import nest_asyncio
from decimal import Decimal

# Apply nest_asyncio to enable async execution in Jupyter
nest_asyncio.apply()

# Add shared directory to import StarknetClient
sys.path.append("../../../apps/")
from shared.starknet_client import StarknetClient

# Initialize Starknet client
starknet_client = StarknetClient()

# VESU contract and pool addresses
VESU_ADDRESS = "0x02545b2e5d519fc230e9cd781046d3a64e092114f07e44771e0d719d148725ef"
POOL_ID = "0x4dc4f0ca6ea4961e4c8373265bfd5317678f4fe374d76f3fd7135f57763bf28"
STRK_ADDRESS = "0x4718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"
USDT_ADDRESS = "0x68f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8"
USER_ID = "0x5a4073a7e24dea4e789be9c1de66f1e38e46204d82cac02959b90ed4056cbc5"

# Token metadata
TOKENS = {
    STRK_ADDRESS: {"symbol": "STRK", "decimals": 18},
    USDT_ADDRESS: {"symbol": "USDT", "decimals": 6},
}

# Cache for expensive operations
_cache = {}


def u256_to_decimal(low, high):
    """Convert u256 (low, high) to Decimal."""
    return Decimal(low) + Decimal(high) * Decimal(2**128)


async def fetch_token_price(address):
    """Fetch token price using the VESU protocol extensions."""
    try:
        # Get pool ID and contract addresses
        vesu_addr = int(VESU_ADDRESS, 16)
        pool_id = int(POOL_ID, 16)
        addr = int(address, 16)

        # Use the cache for extension contract
        cache_key = f"extension_price_{pool_id}"
        if cache_key not in _cache:
            extension_result = await starknet_client.func_call(
                vesu_addr, "extension", [pool_id]
            )
            _cache[cache_key] = extension_result[0]

        price_extension_addr = _cache[cache_key]

        # Use cache for price
        cache_key_price = f"price_{pool_id}_{addr}"
        if cache_key_price not in _cache:
            price_result = await starknet_client.func_call(
                price_extension_addr, "price", [pool_id, addr]
            )

            # Parse result: first value is the price, second indicates if valid
            price_u256 = price_result[0]
            is_valid = price_result[2] if len(price_result) > 2 else 1

            if not is_valid:
                print(
                    f"Warning: Price for {TOKENS.get(address, {}).get('symbol', address)} is not valid"
                )
                return Decimal("1")  # Fallback value

            # Convert from u256 to decimal (assuming 18 decimals)
            price = Decimal(price_u256) / Decimal(10**18)
            _cache[cache_key_price] = price

        return _cache[cache_key_price]

    except Exception as e:
        print(f"Error fetching price for {address}: {e}")
        return Decimal("1")  # Fallback value


async def get_contract_data(cache_key, func, params):
    """Get data from contract with caching."""
    if cache_key not in _cache:
        _cache[cache_key] = await func(*params)
    return _cache[cache_key]


async def get_position_data(user_address, collateral_asset, debt_asset):
    """Get user position data with caching."""
    vesu_addr = int(VESU_ADDRESS, 16)
    pool_id = int(POOL_ID, 16)
    cache_key = f"position_{user_address}_{collateral_asset}_{debt_asset}"
    return await get_contract_data(
        cache_key,
        starknet_client.func_call,
        [vesu_addr, "position", [pool_id, collateral_asset, debt_asset, user_address]],
    )


async def get_asset_config(asset_address):
    """Get asset configuration with caching."""
    vesu_addr = int(VESU_ADDRESS, 16)
    pool_id = int(POOL_ID, 16)
    cache_key = f"asset_config_{asset_address}"
    return await get_contract_data(
        cache_key,
        starknet_client.func_call,
        [vesu_addr, "asset_config", [pool_id, asset_address]],
    )


async def get_ltv_config(collateral_asset, debt_asset):
    """Get LTV configuration with caching."""
    vesu_addr = int(VESU_ADDRESS, 16)
    pool_id = int(POOL_ID, 16)
    cache_key = f"ltv_{collateral_asset}_{debt_asset}"
    return await get_contract_data(
        cache_key,
        starknet_client.func_call,
        [vesu_addr, "ltv_config", [pool_id, collateral_asset, debt_asset]],
    )


async def get_utilization(asset_address):
    """Get asset utilization with caching."""
    vesu_addr = int(VESU_ADDRESS, 16)
    pool_id = int(POOL_ID, 16)
    asset_addr = int(asset_address, 16)
    cache_key = f"utilization_{asset_addr}"
    
    if cache_key not in _cache:
        utilization_data = await starknet_client.func_call(
            vesu_addr, "utilization", [pool_id, asset_addr]
        )
        _cache[cache_key] = Decimal(utilization_data[0]) / Decimal(10**18)
        
    return _cache[cache_key]


async def get_rate_accumulator(asset_address):
    """Get rate accumulator with caching."""
    vesu_addr = int(VESU_ADDRESS, 16)
    pool_id = int(POOL_ID, 16)
    asset_addr = int(asset_address, 16)
    cache_key = f"rate_acc_{asset_addr}"
    
    if cache_key not in _cache:
        rate_acc_data = await starknet_client.func_call(
            vesu_addr, "rate_accumulator", [pool_id, asset_addr]
        )
        _cache[cache_key] = rate_acc_data
        
    return _cache[cache_key]


async def calculate_collateral_value(pool_id, asset, shares_low, shares_high, sign=0):
    """Calculate collateral value."""
    vesu_addr = int(VESU_ADDRESS, 16)
    cache_key = f"collateral_{asset}_{shares_low}_{shares_high}_{sign}"
    result = await get_contract_data(
        cache_key,
        starknet_client.func_call,
        [
            vesu_addr,
            "calculate_collateral",
            [pool_id, asset, shares_low, shares_high, sign],
        ],
    )
    return u256_to_decimal(result[0], result[1])


async def calculate_debt_value(
    nominal_debt_low,
    nominal_debt_high,
    sign,
    rate_accumulator_low,
    rate_accumulator_high,
    scale_low,
    scale_high,
):
    """Calculate debt value."""
    vesu_addr = int(VESU_ADDRESS, 16)
    cache_key = f"debt_{nominal_debt_low}_{nominal_debt_high}_{sign}_{rate_accumulator_low}_{rate_accumulator_high}"
    result = await get_contract_data(
        cache_key,
        starknet_client.func_call,
        [
            vesu_addr,
            "calculate_debt",
            [
                nominal_debt_low,
                nominal_debt_high,
                sign,
                rate_accumulator_low,
                rate_accumulator_high,
                scale_low,
                scale_high,
            ],
        ],
    )
    return u256_to_decimal(result[0], result[1])

## 1. Health Factor Calculation

The health factor measures the safety of a position against liquidation, calculated as:

```
Health Factor = (Collateral Value in USD * Collateral Factor) / Borrow Value in USD
```

A value > 1 indicates a safe position, while < 1 signals liquidation risk. This section fetches collateral (STRK) and debt (USDT) data for a specific pool and computes the health factor, using the STRK/USDT price from MySwap.

In [26]:
async def calculate_health_factor():
    """Calculate health factor for STRK/USDT pair for a user in a pool."""
    pool_id = int(POOL_ID, 16)
    collateral_asset = int(STRK_ADDRESS, 16)
    debt_asset = int(USDT_ADDRESS, 16)
    user = int(USER_ID, 16)

    # Fetch user position data
    position_data = await get_position_data(user, collateral_asset, debt_asset)
    collateral_shares_low = position_data[0]
    collateral_shares_high = position_data[1]
    collateral_shares_sign = 0 if collateral_shares_low >= 0 else 1

    # Calculate collateral value
    collateral_value = await calculate_collateral_value(
        pool_id,
        collateral_asset,
        collateral_shares_low,
        collateral_shares_high,
        collateral_shares_sign,
    )

    # Get debt configuration and calculate debt
    debt_config = await get_asset_config(debt_asset)
    nominal_debt_low = position_data[2]
    nominal_debt_high = position_data[3]
    nominal_debt_sign = 0 if nominal_debt_low >= 0 else 1
    rate_accumulator_low = debt_config[14]
    rate_accumulator_high = debt_config[15]
    scale_low = debt_config[10]
    scale_high = debt_config[11]

    # Calculate debt value
    debt_value = await calculate_debt_value(
        nominal_debt_low,
        nominal_debt_high,
        nominal_debt_sign,
        rate_accumulator_low,
        rate_accumulator_high,
        scale_low,
        scale_high,
    )

    # Get LTV configuration
    ltv_data = await get_ltv_config(collateral_asset, debt_asset)
    collateral_factor = Decimal(ltv_data[0]) / Decimal(10**18)

    # Get prices
    strk_price = await fetch_token_price(STRK_ADDRESS)
    usdt_price = await fetch_token_price(USDT_ADDRESS)

    # Normalize and calculate USD values
    collateral_value_normalized = collateral_value / Decimal(10**18)
    debt_value_normalized = debt_value / Decimal(10**6)
    collateral_value_usd = collateral_value_normalized * strk_price
    debt_value_usd = debt_value_normalized * usdt_price

    # Calculate health factor
    health_factor = (
        (collateral_value_usd * collateral_factor) / debt_value_usd
        if debt_value_usd > 0
        else Decimal("inf")
    )

    print(f"Health Factor for user {USER_ID}: {health_factor:.4f}")
    return (
        health_factor,
        collateral_value_normalized,
        debt_value_normalized,
        collateral_factor,
    )


async def analyze_health_factor():
    """Analyze health factor for the STRK/USDT pair in a pool."""
    health_factor, _, _, _ = await calculate_health_factor()
    print(f"Pool Health Factor: {health_factor:.4f}")

    if health_factor > 1.5:
        risk_level = "Low Risk"
        recommendation = "Position is well-collateralized"
    elif health_factor > 1.2:
        risk_level = "Moderate Risk"
        recommendation = "Consider adding more collateral for safety"
    elif health_factor > 1.0:
        risk_level = "High Risk"
        recommendation = "Position at risk of liquidation if market moves against you"
    else:
        risk_level = "Liquidation Risk"
        recommendation = "Position is subject to liquidation"

    print(f"Risk Level: {risk_level}")
    print(f"Recommendation: {recommendation}")

## 2. Interest Rate Mechanism

VESU employs a dynamic interest rate model based on asset utilization:

- `Utilization Rate = Total Borrows / Total Supply`
- `Borrow Rate = Base Rate + (Utilization * Multiplier)`
- `Supply Rate = Borrow Rate * Utilization * (1 - Reserve Factor)`

This section calculates borrow and supply rates for STRK in a specific pool.

In [27]:
async def analyze_interest_rates():
    """Calculate borrow and supply rates for STRK in a pool."""
    pool_id = int(POOL_ID, 16)
    asset_addr = int(STRK_ADDRESS, 16)

    # Get asset configuration
    config = await get_asset_config(asset_addr)
    
    # Extract relevant data
    total_collateral_shares_low = config[0]
    total_collateral_shares_high = config[1]
    total_nominal_debt_low = config[2]
    total_nominal_debt_high = config[3]
    asset_scale_low = config[10]
    asset_scale_high = config[11]

    # Calculate supply and borrow
    total_supply = await calculate_collateral_value(
        pool_id, asset_addr, total_collateral_shares_low, total_collateral_shares_high
    )

    # Get utilization directly from contract
    utilization = await get_utilization(STRK_ADDRESS)

    # Get rate accumulator
    rate_acc_data = await get_rate_accumulator(STRK_ADDRESS)
    rate_accumulator = Decimal(rate_acc_data[0]) / Decimal(10**18)

    total_borrows = await calculate_debt_value(
        total_nominal_debt_low,
        total_nominal_debt_high,
        0,
        rate_acc_data[0],
        rate_acc_data[1],
        asset_scale_low,
        asset_scale_high,
    )

    # Convert to token units
    asset_scale = u256_to_decimal(asset_scale_low, asset_scale_high)
    total_supply_tokens = total_supply / asset_scale
    total_borrows_tokens = total_borrows / asset_scale

    # Interest rate model parameters (typical values, may differ from actual protocol)
    base_rate = Decimal("0.01")  # 1% base interest rate
    multiplier = Decimal("0.05")  # 5% slope parameter
    reserve_factor = Decimal("0.1")  # 10% of interest goes to protocol reserves
    
    # For more accurate results, these values should be fetched from the protocol's
    # interest rate model contract or configuration

    # Calculate APYs
    borrow_rate = base_rate + utilization * multiplier
    supply_rate = borrow_rate * utilization * (1 - reserve_factor)

    print(f"Total Supply (STRK): {total_supply_tokens:.2f}")
    print(f"Total Borrows (STRK): {total_borrows_tokens:.2f}")
    print(f"Rate Accumulator: {rate_accumulator:.6f}")
    print(f"STRK Utilization Rate: {utilization*100:.2f}%")
    print(f"STRK Borrow APY: {borrow_rate*100:.2f}%")
    print(f"STRK Supply APY: {supply_rate*100:.2f}%")

## 3. Deposit Functionality

This section verifies deposit status and total supply for STRK and USDT in a specific pool.

In [28]:
async def analyze_deposit_functionality():
    """Check deposit support and total supply for assets in the pool."""
    pool_id = int(POOL_ID, 16)

    for addr_str, info in TOKENS.items():
        addr = int(addr_str, 16)
        config = await get_asset_config(addr)

        # Extract shares data
        total_collateral_shares_low = config[0]
        total_collateral_shares_high = config[1]

        # Calculate total supply
        total_supply = await calculate_collateral_value(
            pool_id, addr, total_collateral_shares_low, total_collateral_shares_high
        )

        # Normalize using token decimals
        asset_scale_low = config[10]
        asset_scale_high = config[11]
        asset_scale = u256_to_decimal(asset_scale_low, asset_scale_high)
        total_supply_tokens = total_supply / asset_scale

        print(f"Total Supply for {info['symbol']}: {total_supply_tokens:,.2f}")

## 4. Borrowing Potential

This section calculates the borrowing capacity for the STRK/USDT pair:

- `Max Borrowable (USDT) = (Collateral Value in USD * Collateral Factor)`
- `Additional Borrowing Power = Max Borrowable - Current Borrowed`

In [29]:
async def analyze_borrowing_potential():
    """Analyze borrowing potential for the STRK/USDT pair."""
    # Most data already computed in calculate_health_factor
    (
        _,
        collateral_value_normalized,
        current_borrowed_usdt,
        collateral_factor,
    ) = await calculate_health_factor()

    # Get STRK price
    strk_price = await fetch_token_price(STRK_ADDRESS)

    # Calculate borrowing power
    collateral_value_usd = collateral_value_normalized * strk_price
    max_borrowable_usdt = collateral_value_usd * collateral_factor
    additional_borrowing_power_usdt = max(
        Decimal(0), max_borrowable_usdt - current_borrowed_usdt
    )

    print("\nSTRK-USDT Borrowing Analysis")
    print("==========================")
    print(f"STRK Collateral: {collateral_value_normalized:,.2f} STRK")
    print(f"Collateral Value USD: {collateral_value_usd:,.2f} USD")
    print(f"Collateral Factor: {collateral_factor:.2%}")
    print(f"Current USDT Borrowed: {current_borrowed_usdt:,.2f} USDT")
    print(f"Maximum Borrowable USDT: {max_borrowable_usdt:,.2f} USDT")
    print(f"Additional USDT Available: {additional_borrowing_power_usdt:,.2f} USDT")

## Summary

This notebook provides a comprehensive analysis of the VESU protocol for the STRK/USDT pair:

1. **Health Factor**: Calculated safety against liquidation.
2. **Interest Rates**: Modeled dynamically for STRK based on utilization.
3. **Deposits**: Verified status and supply for STRK and USDT.
4. **Borrowing Potential**: Assessed maximum and additional capacity.

**Note**: Assumptions about struct indices and MySwap pool IDs should be validated with VESU ABI and MySwap documentation.

In [30]:
async def analyze_all():
    """Run all analyses with error handling."""
    try:
        print("Running Health Factor Analysis...")
        await analyze_health_factor()

        print("\nRunning Interest Rates Analysis...")
        await analyze_interest_rates()

        print("\nRunning Deposit Functionality Analysis...")
        await analyze_deposit_functionality()

        print("\nRunning Borrowing Potential Analysis...")
        await analyze_borrowing_potential()
    except Exception as e:
        import traceback

        print(f"Error during analysis: {e}")
        traceback.print_exc()


# Run all analyses
await analyze_all()

Running Health Factor Analysis...
Health Factor for user 0x5a4073a7e24dea4e789be9c1de66f1e38e46204d82cac02959b90ed4056cbc5: 1.9103
Pool Health Factor: 1.9103
Risk Level: Low Risk
Recommendation: Position is well-collateralized

Running Interest Rates Analysis...
Total Supply (STRK): 4130155.02
Total Borrows (STRK): 230677.34
Rate Accumulator: 1.000807
STRK Utilization Rate: 5.59%
STRK Borrow APY: 1.28%
STRK Supply APY: 0.06%

Running Deposit Functionality Analysis...
Total Supply for STRK: 4,130,155.02
Total Supply for USDT: 355,537.18

Running Borrowing Potential Analysis...
Health Factor for user 0x5a4073a7e24dea4e789be9c1de66f1e38e46204d82cac02959b90ed4056cbc5: 1.9103

STRK-USDT Borrowing Analysis
STRK Collateral: 3,720.00 STRK
Collateral Value USD: 658.42 USD
Collateral Factor: 58.00%
Current USDT Borrowed: 200.00 USDT
Maximum Borrowable USDT: 381.88 USDT
Additional USDT Available: 181.88 USDT
